In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import sqlite3

In [ ]:
#get all product
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(30)
driver.maximize_window()
driver.get('https://www.tokopedia.com/gudangjaket/page/2')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

elems = driver.find_elements_by_xpath("//a")
data = []
for elem in elems:
    
    data_test = elem.get_attribute("data-testid")
    if data_test == "lnkProductContainer":
        href = elem.get_attribute("href")
        product_name = elem.text
        product_name = product_name.replace('Produk Unggulan\n','')
        product_name = product_name.replace('Stok Kosong\n','')
        products = product_name.split('\n')
        row = [href,products[0],products[1]]
        data.append(row)

driver.quit()
df = pd.DataFrame(data,columns=['href','name','price'])
# df.head(n=10)
# df.to_csv(r'toped_master_data.csv', index = False, header=True, doublequote=True)
conn = sqlite3.connect('scrap.gudangjaket.db')
df.to_sql('GUDANGJAKET_MASTER', conn, if_exists='append', index = False)

In [ ]:
conn = sqlite3.connect('scrap.gudangjaket.db')
cur = conn.cursor()
cur.execute('select href from GUDANGJAKET_MASTER')
reader = cur.fetchall()
data = []
for line in reader:
    print(line[0])

In [ ]:
#grab detail
conn = sqlite3.connect('scrap.gudangjaket.db')
cur = conn.cursor()
cur.execute('select href from GUDANGJAKET_MASTER limit 1')
reader = cur.fetchall()
data = []
for line in reader:
    # print(line[0])
    driver = webdriver.Chrome('chromedriver')
    driver.implicitly_wait(30)
    driver.maximize_window()
    driver.get(line[0])
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    #find title
    elems = driver.find_elements_by_xpath("//h1")
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "lblPDPDetailProductName":
            product_title = elem.text
    
    #find price
    elems = driver.find_elements_by_xpath("//h3")
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "lblPDPDetailProductPrice":
            product_price = elem.text

    #find price
    elems = driver.find_elements_by_xpath("//img")
    for elem in elems:
        data_test = elem.get_attribute("alt")
        if data_test == "product image":
            product_image = elem.get_attribute("src")
    
    #find description
    elems = driver.find_elements_by_xpath("//p")
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "lblPDPDeskripsiProduk":
            product_desc = elem.text
        if data_test == "PDPDetailWeightValue":
            product_weight = elem.text

    #find variant
    variants = []
    elems = driver.find_elements_by_xpath("//div")
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "lblPDPProductVariantukuran":
            variants.append(elem.text)
        if data_test == "lblPDPProductVariantwarna":
            variants.append(elem.text)

    row = [product_title, product_price, product_weight, product_image, product_desc]
    data.append(row)
    driver.quit()
    time.sleep(2)

# print(data)
# print(variant)

uploads = []
sizes = []
colour = []
for variant in variants:
    # print(variant)
    if variant == "L" or variant == "XL":
        sizes.append(variant)
    else:
        colour.append(variant)

for d in data:
    for size in sizes:
        for c in colour:
            rows = [d[0],c,size,d[1],d[2],d[3],d[4]]
            uploads.append(rows)

# print(uploads)
df = pd.DataFrame(uploads,columns=['title','colour','size','price','weight','image','desc'])
# df.to_csv(r'toped_final.csv', index = False, header=True, doublequote=True)
conn = sqlite3.connect('scrap.gudangjaket.db')
df.to_sql('GUDANGJAKET_SESI3', conn, if_exists='replace', index = False)

In [ ]:

uploads = []
sizes = []
colour = []
for variant in variants:
    # print(variant)
    if variant == "L" or variant == "XL":
        sizes.append(variant)
    else:
        colour.append(variant)

for d in data:
    for size in sizes:
        for c in colour:
            rows = [d[0],c,size,d[1],d[2],d[3],d[4]]
            uploads.append(rows)

# print(uploads)
df = pd.DataFrame(uploads,columns=['title','colour','size','price','weight','image','desc'])
# df.to_csv(r'toped_final.csv', index = False, header=True, doublequote=True)
conn = sqlite3.connect('scrap.gudangjaket.db')
df.to_sql('GUDANGJAKET_SESI3', conn, if_exists='replace', index = False)